# Вебинар 6. Двухуровневые модели рекомендаций


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
#from src.recommenders import MainRecommender - как она прописана в вашем модуле?

from scipy.sparse import csr_matrix, coo_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

/Users/ekaterinarazdumina/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv('./retail_train.csv')
item_features = pd.read_csv('./product.csv')
user_features = pd.read_csv('./hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

#параметры функции могут отличаться
data_train_lvl_1 = prefilter_items(data_train_lvl_1, filter='no_nopop')


n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


/Users/ekaterinarazdumina/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
result = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [5]:
# your_code

In [6]:
user_item_matrix = pd.pivot_table(data_train_lvl_1, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819255,...,15511891,15596279,15596488,15596515,15778533,15926844,15926886,15927403,15927661,15927850
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [9]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 3.96 s, sys: 141 ms, total: 4.1 s
Wall time: 3.87 s


In [10]:
result = result.drop(result[(result['user_id'] == 1984)].index)

In [11]:
def get_recommendations(user, model, N=50):
        res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None,  
                                    recalculate_user=True)]
        return res

In [12]:
%%time
    
result['bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=50))

CPU times: user 1min 11s, sys: 11.4 s, total: 1min 23s
Wall time: 21.1 s


In [13]:
user_item_matrix = pd.pivot_table(data_train_lvl_1, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819255,...,15511891,15596279,15596488,15596515,15778533,15926844,15926886,15927403,15927661,15927850
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
%%time

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)


CPU times: user 11.8 s, sys: 1.23 s, total: 13 s
Wall time: 5 s


In [16]:
def get_recommendations(user, model, N=50):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [17]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=50))

CPU times: user 1min 11s, sys: 10.5 s, total: 1min 22s
Wall time: 21.2 s


In [18]:
result.apply(lambda row: recall_at_k(row['als'], row['actual']), axis=1).mean()

0.028571354019628944

In [19]:
result.apply(lambda row: recall_at_k(row['bm25'], row['actual']), axis=1).mean()

0.015112665539587001

In [20]:
result.head()

,user_id,actual,bm25,als
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1104349, 1082212, 1004390, 1094924, 1062572, ...","[1033142, 1056509, 995242, 9527290, 1029743, 9..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1007195, 908846, 1041259, 5569230, 916122, 99...","[1106523, 1133018, 999999, 5569230, 1082185, 9..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67...","[891423, 902172, 1082627, 846550, 1021324, 809...","[999999, 1082185, 962229, 883932, 951590, 1029..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[1082185, 878996, 965267, 930118, 1024306, 863...","[1023720, 1007195, 923746, 866211, 878996, 103..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886...","[853643, 857390, 912553, 1130111, 1003188, 999...","[999999, 1082185, 1058997, 1013321, 938700, 89..."


### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар 
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [21]:
# your_code

In [22]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: get_recommendations(x, model=model, N=50))

In [23]:
users_lvl_2_val = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2_val.columns = ['user_id']

test_users = data_train_lvl_1['user_id'].unique()
users_lvl_2_val = users_lvl_2_val[users_lvl_2_val['user_id'].isin(test_users)]

users_lvl_2_val['candidates'] = users_lvl_2_val['user_id'].apply(lambda x: get_recommendations(x, model=model, N=50))

In [24]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [25]:
df_val=pd.DataFrame({'user_id':users_lvl_2_val.user_id.values.repeat(len(users_lvl_2_val.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2_val.candidates.values)})

In [26]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [27]:
targets_lvl_2_val = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2_val['target'] = 1  # тут только покупки 

targets_lvl_2_val = df_val.merge(targets_lvl_2_val, on=['user_id', 'item_id'], how='left')

targets_lvl_2_val['target'].fillna(0, inplace= True)

In [28]:
targets_lvl_2.head(10)

,user_id,item_id,target
0,2070,1038217,0.0
1,2070,908531,1.0
2,2070,981760,1.0
3,2070,1098066,0.0
4,2070,826249,0.0
5,2070,863802,1.0
6,2070,962568,1.0
7,2070,1096036,0.0
8,2070,1133018,0.0
9,2070,896862,0.0


In [29]:
average_check = data.groupby('user_id')['sales_value'].sum()/(data.groupby('user_id')['sales_value'].count())
average_check = pd.DataFrame(data=average_check)

frequency = data.groupby('user_id')['sales_value'].count() / ((data.groupby('user_id')['week_no'].max() - data.groupby('user_id')['week_no'].min()))
frequency = pd.DataFrame(data=frequency)

item_value = data.groupby('item_id')['sales_value'].sum()/data.groupby('item_id')['sales_value'].count()
item_value = pd.DataFrame(data=item_value)

In [30]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(average_check, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(frequency, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(item_value, on='item_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,sales_value_x,0,sales_value_y
0,2070,1038217,0.0,69.0,GROCERY,Private,CRACKERS/MISC BKD FD,SOUP CRACKERS (SALTINE/OYSTER),16 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,29.814815,1.158960
1,2070,908531,1.0,69.0,GROCERY,Private,FLUID MILK PRODUCTS,CHOCOLATE MILK,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,29.814815,1.742008


In [31]:
targets_lvl_2_val = targets_lvl_2_val.merge(item_features, on='item_id', how='left')
targets_lvl_2_val = targets_lvl_2_val.merge(user_features, on='user_id', how='left')
targets_lvl_2_val = targets_lvl_2_val.merge(average_check, on='user_id', how='left')
targets_lvl_2_val = targets_lvl_2_val.merge(frequency, on='user_id', how='left')
targets_lvl_2_val = targets_lvl_2_val.merge(item_value, on='item_id', how='left')

In [32]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

X_test = targets_lvl_2_val.drop('target', axis=1)
y_test = targets_lvl_2_val[['target']]

In [33]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')
X_test[cat_feats] = X_test[cat_feats].astype('category')

In [34]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)
test_preds = lgb.predict(X_test)

/Users/ekaterinarazdumina/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/ekaterinarazdumina/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


In [35]:
X_test['result'] = test_preds

In [36]:
result_val_2 = X_test[['user_id', 'item_id', 'result']]
result_val_2 = result_val_2.loc[result_val_2['result']==1]
result_val_2 = result_val_2.drop('result', axis=1)

In [42]:
results = []

In [37]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.1964700418021334

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 0.27%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 